In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/MSBA 7011 Group Project/Clothes-Detection')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch
import os
import cv2
from yolo.utils.utils import *
from predictors.YOLOv3 import YOLOv3Predictor
# from predictors.DetectronModels import Predictor
import glob
from tqdm import tqdm
import sys
import tqdm

# Preparation

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

#YOLO PARAMS
yolo_df2_params = {   "model_def" : "yolo/df2cfg/yolov3-df2.cfg",
"weights_path" : "yolo/weights/yolov3-df2_15000.weights",
"class_path":"yolo/df2cfg/df2.names",
"conf_thres" : 0.5,
"nms_thres" :0.4,
"img_size" : 416,
"device" : device}

#DATASET
dataset = 'df2'
yolo_params = yolo_df2_params

#Classes
classes = load_classes(yolo_params["class_path"])

# #Colors
# cmap = plt.get_cmap("rainbow")
# colors = np.array([cmap(i) for i in np.linspace(0, 1, 13)])
# #np.random.shuffle(colors)

model = 'yolo'

if model == 'yolo':
    detectron = YOLOv3Predictor(params=yolo_params)
else:
    detectron = Predictor(model=model,dataset= dataset, CATEGORIES = classes)

Model loaded successfully from yolo/weights/yolov3-df2_15000.weights.


# Read files

In [0]:
# input the folder of IG images that you want to detect
folder = input('input the folder of IG images: ')
# folder = 'fashion4perfection'
WSI_MASK_PATH = '../data/IG/c-{}/'.format(folder)

input the folder of IG images: fashion4perfection


In [0]:
paths = glob.glob(os.path.join(WSI_MASK_PATH,'*.jpg'))
paths.sort()

In [0]:
paths[:5]
# !ls

['../data/IG/c-fashion4perfection/2018-01-01_10-12-16_UTC.jpg',
 '../data/IG/c-fashion4perfection/2018-01-01_12-17-01_UTC.jpg',
 '../data/IG/c-fashion4perfection/2018-01-01_12-55-17_UTC.jpg',
 '../data/IG/c-fashion4perfection/2018-01-01_16-02-38_UTC.jpg',
 '../data/IG/c-fashion4perfection/2018-01-01_23-22-37_UTC.jpg']

In [0]:
len(paths)

10789

# Detection 

In [0]:
count = 8000
valid_img_count = 1465


for path in paths[8000:]:
  count += 1 
  # print("image:",str(count))
  img = cv2.imread(path)
  img_id = path.split('/')[-1].split('.')[0]
  detections = detectron.get_detections(img)

  item_count = 0
  # if len(detections) != 2:
    # print('not enough item detected')

  # elif len(detections) == 2:
  if len(detections) == 2:
    valid_img_count += 1
    detections.sort(reverse=False ,key = lambda x:x[4])
    for x1, y1, x2, y2, cls_conf, cls_pred in detections:
      item_count+=1

      if cls_pred in range(6):
        cls_name = 'top'
        print("image:{}, item{}:{}-{}, conf:{}".format(count,item_count, cls_name, classes[int(cls_pred)], round(cls_conf,3))) 
      elif cls_pred in range(6,9):
        cls_name = 'bottom'
        print("image:{}, item{}:{}-{}, conf:{}".format(count,item_count, cls_name, classes[int(cls_pred)], round(cls_conf,3))) 
      else:
        cls_name = 'other'
        print('item{}, other category detected, {}; id={}:'.format(item_count, classes[int(cls_pred)], img_id))
          
      if cls_name == 'top' or cls_name == 'bottom':
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        y1 = 0 if y1<0 else y1
        # crop image and output
        cropped = img[y1:y2, x1:x2]  
        file_name = "{}-{}_{}".format(folder, valid_img_count, cls_name)
        try:
          cv2.imwrite('../413_triplet/new_output/{}.jpg'.format(file_name), cropped)
          cv2.waitKey(0)
        except:
          print("failed to write file")
      # cv2.waitKey(0)

    # cv2.waitKey(0)

print('valid images: {}/{}'.format(valid_img_count, count))

item1, other category detected, vest dress; id=2019-07-21_16-13-39_UTC:
image:8007, item2:top-sling, conf:0.669
image:8011, item1:top-long sleeve top, conf:0.804
image:8011, item2:bottom-trousers, conf:0.804
image:8014, item1:bottom-trousers, conf:0.891
image:8014, item2:top-short sleeve top, conf:0.955
item1, other category detected, vest dress; id=2019-07-25_09-44-21_UTC:
image:8046, item2:top-sling, conf:0.662
image:8048, item1:bottom-shorts, conf:0.658
item2, other category detected, sling dress; id=2019-07-25_13-50-45_UTC:
image:8054, item1:top-sling, conf:0.891
image:8054, item2:bottom-shorts, conf:0.996
image:8064, item1:bottom-shorts, conf:0.752
image:8064, item2:top-short sleeve top, conf:0.953
image:8069, item1:bottom-skirt, conf:0.604
image:8069, item2:top-long sleeve top, conf:0.75
image:8078, item1:top-vest, conf:0.849
image:8078, item2:bottom-shorts, conf:0.971
image:8083, item1:top-long sleeve top, conf:0.875
image:8083, item2:top-long sleeve outwear, conf:0.934
image:80